In [52]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import psycopg2
import scraper
import process_words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

In [53]:
conn = psycopg2.connect(dbname='food_db')
df = pd.read_sql('SELECT * FROM recipes', con=conn)

In [169]:
top_words, label_list = process_words.make_top_words(df, 'foods', 20)

In [170]:
top_words

,2009_1,s2009_1,2009_2,s2009_2,2009_3,s2009_3,2009_4,s2009_4,2010_1,s2010_1,...,2018_17,s2018_17,2018_18,s2018_18,2018_19,s2018_19,2018_20,s2018_20,2018_21,s2018_21
0,oil,0.112582,butter,0.079584,egg,0.188409,chicken,0.155446,nut,0.175772,...,pomegran,0.542747,half,0.301218,cucumb,0.401554,fish,0.533440,haricot,0.292787
1,tomato,0.094246,juic,0.073942,powder,0.162482,mustard,0.100574,pine,0.164861,...,orang,0.428420,sprig,0.282861,flake,0.299136,wine,0.398714,vert,0.292787
2,clove,0.083140,sugar,0.068926,sugar,0.151962,appl,0.075061,beet,0.103152,...,grapefruit,0.361832,pea,0.191812,chili,0.168784,marsala,0.266720,fennel,0.292787
3,onion,0.082706,potato,0.056002,butter,0.121545,oil,0.072764,oil,0.078085,...,flower,0.180916,kosher,0.149123,vinegar,0.122092,snapper,0.266720,bulb,0.292787
4,vinegar,0.048546,water,0.048793,vanilla,0.115941,part,0.069186,green,0.074109,...,creme,0.180916,onion,0.112493,sugar,0.118075,fishmong,0.266720,radish,0.292787
5,lemon,0.039792,oil,0.043581,extract,0.095655,stock,0.067771,mint,0.057374,...,tangerin,0.180916,clove,0.110409,bunch,0.109678,bass,0.266720,frond,0.292787
6,wine,0.035686,cream,0.043502,milk,0.094685,shallot,0.055948,clove,0.054295,...,aril,0.180916,split,0.110211,mirin,0.109678,rock,0.266720,purpl,0.292787
7,cumin,0.035574,cinnamon,0.042788,soda,0.064793,kosher,0.052739,onion,0.053968,...,blossom,0.180916,stuff,0.110211,takara,0.109678,segment,0.241858,dijon,0.231112
8,virgin,0.035046,stick,0.042463,chocol,0.064773,cream,0.051724,parsley,0.049484,...,envelop,0.180916,paprika,0.110211,noodl,0.109678,stalk,0.224219,mustard,0.231112
9,coriand,0.034017,mint,0.040805,appl,0.063527,dijon,0.049162,hand,0.046820,...,fraich,0.180916,pastri,0.105310,shichimi,0.109678,ginger,0.189905,orang,0.231112


In [185]:
r_type='appetizers'
response = requests.get("https://food52.com/recipes/{}?".format(r_type))
soup = BeautifulSoup(response.text, "html.parser")
pages = soup.find("div", {"class" : "pagination"})
pg = int(pages.find_all("a")[-2].text)

In [186]:
pg

317

In [179]:
response

<Response [404]>